In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import time
import types
sys.path.append('../')
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, interactive, interactive_output
from traitlets import traitlets
import matplotlib.pyplot as plt
from fbprophet import Prophet
import yaml
from src.data_downloader import DATA_REPOS, download_from_repo, get_dataframes
import plotly.graph_objects as go
from plotly.graph_objs import Layout
from plotly.subplots import make_subplots
import plotly.express as px
from IPython.display import display, clear_output
with open("columns_names.yaml", 'r') as stream:
    out = yaml.load(stream)
    orig_data_columns = out['LABELS']['orig_data_columns']
    extra_data_columns = out['LABELS']['extra_data_columns']
    prov_data_columns = out['LABELS']['prov_data_columns']
    trend_labels = out['LABELS']['trend_labels']
import warnings
warnings.filterwarnings('ignore')
dest='../data'

/home/maurizio/anaconda3/envs/covid19/lib/python3.7/site-packages/ipykernel_launcher.py:23: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



In [2]:
df_naz, reg, prov, df_reg, df_prov, df_world_confirmed, df_world_deaths, df_world_recovered, populations, ita_populations, df_comuni_sett = get_dataframes(dest, npt_rth=5, smooth=True)

last available date for Italy data 2020-06-02T17:00:00
last available date for World data 6/1/20


In [3]:
# first, set the main page as a Tab container
main_tab = widgets.Tab()
main_tab.set_title(0, 'Italy')

In [4]:
# first tab is for data download
download_tab = widgets.VBox()
download_out = widgets.Output(layout={'border': '1px solid black'})

@download_out.on_displayed
def check_creation_date(b=None):
    file1 = 'dpc-covid19-ita-regioni.csv'
    file2 = 'time_series_covid19_confirmed_global.csv'
    file_path1 = os.path.join(dest,file1)
    file_path2 = os.path.join(dest,file2)
    (mode, ino, dev, nlink, uid, gid, size, atime, mtime1, ctime) = os.stat(file_path1)
    (mode, ino, dev, nlink, uid, gid, size, atime, mtime2, ctime) = os.stat(file_path2)
    download_out.append_display_data("{} last modified: {}".format(file1, time.ctime(mtime1)))
    download_out.append_display_data("{} last modified: {}".format(file2, time.ctime(mtime2)))
    

update_button = widgets.Button(description='Update data from repos')
@update_button.on_click
def update_repo(b=None):
    download_out.append_display_data('downloading Italian data')
    download_from_repo(DATA_REPOS['italy']['url'], filenames=DATA_REPOS['italy']['streams'], dest=dest)
    download_out.append_display_data('downloading world data')
    download_from_repo(DATA_REPOS['world']['url'], filenames=DATA_REPOS['world']['streams'], dest=dest)  
    df_naz, reg, prov, df_reg, df_prov, df_world_confirmed, df_world_deaths, df_world_recovered, populations, ita_populations, df_comuni_sett = get_dataframes(dest, npt_rth=5, smooth=True)


download_tab.children = [widgets.HBox(children=[download_out, update_button])]

In [5]:
# second tab for italy
italy_tab = widgets.Tab()

In [6]:
prov_tab = widgets.VBox()
province_out = widgets.Output(layout = {
            'width': '100%',
            'height': '600px',
            'border': '1px solid black'
        })

def get_top_provinces(label, top_prov,date, show_map, show_grid):    
    with province_out:
        clear_output()
        label = [label]
        df_prov.index = pd.to_datetime(df_prov.index)
        tempdf = df_prov.loc[str(date)][['sigla_provincia','denominazione_provincia', 'lat', 'long']+ label].sort_values(by=label, 
             ascending=False)[:top_prov].set_index('sigla_provincia')

        if show_map:
            fig = px.choropleth(tempdf, 
                                geojson='https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson', 
                                locations='denominazione_provincia', 
                                color=label[0], 
                                color_continuous_scale='Reds',
                                featureidkey='properties.prov_name',                                   
                                range_color=(0, max(tempdf[label[0]])))
            
            fig.update_layout(showlegend=True,title='italian provinces',
                              paper_bgcolor='rgba(0,0,0,0)',font = dict(color = 'lightgray'),plot_bgcolor='rgba(0,0,0,0)')
            fig.update_geos(resolution=50, showcountries=True, countrycolor="lightgray", showland=True, 
                            showsubunits=True, subunitcolor="Blue", #showocean=True, oceancolor="lightblue"
                           )
            fig.update_geos(fitbounds="locations", visible=False)
            fig.update_layout(height=600)
        else:
            fig = px.bar(tempdf[label].reset_index(), x=label[0], y='sigla_provincia', orientation='h')
            fig.update_layout(showlegend=True,title='top {} provinces on day {}'.format(top_prov, date.strftime("%m/%d/%Y")),
                             paper_bgcolor='rgba(0,0,0,0)',font = dict(color = 'lightgray'),plot_bgcolor='rgba(0,0,0,0)')
            fig.update_xaxes(showgrid=show_grid, gridwidth=1, gridcolor='gray')
            fig.update_yaxes(showgrid=show_grid, gridwidth=1, gridcolor='gray')
        fig.show()
        
prov_widget = interactive(get_top_provinces,label=widgets.Select(options=prov_data_columns),
                            top_prov=widgets.IntSlider(min=1,max=150,step=1,value=150),
                            date=widgets.DatePicker(description='Pick a Date',value=pd.to_datetime(df_prov.index.max())),
                            show_map=widgets.Checkbox(value=True),
                            show_grid=widgets.Checkbox(value=True),
                            continuous_update=False
                            )
prov_box = widgets.HBox(prov_widget.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))
prov_tab.children = [prov_box, province_out]

In [7]:
forecast_tab = widgets.VBox()
forecast_out = widgets.Output(layout = {
            'width': '100%',
            'height': '600px',
            'border': '1px solid black'
        })

def get_forecast(region,start_fit,end_fit,label,forecast_periods,smoothing):
    with forecast_out:
        clear_output()
        df = df_reg[region][label].rolling(smoothing).mean()
        y = label
        train_data = pd.DataFrame()
        train_data['ds']=pd.to_datetime(df.index)
        train_data['y']=np.log1p(df.reset_index(drop=True).values)
        train_data['floor'] = 0.
        m = Prophet(growth='linear', daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=False)
        m.fit(train_data.set_index('ds').loc[start_fit:end_fit].reset_index())
        future = m.make_future_dataframe(periods=forecast_periods)
        future['floor'] = train_data['floor']
        forecast = m.predict(future)
        forecast['yhat'] = np.expm1(forecast['yhat'])
        forecast['yhat_lower'] = np.expm1(forecast['yhat_lower'])
        forecast['yhat_upper'] = np.expm1(forecast['yhat_upper'])
        train_data['y'] = np.expm1(train_data['y'])
        df = pd.merge(left=train_data, right=forecast, on='ds', how='outer').set_index('ds')
        df.index = pd.to_datetime(df.index)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df.index, y=df['yhat_lower'],fill=None,mode='lines',line_color='lightgrey',name='confidence_lev_down'))
        fig.add_trace(go.Scatter(x=df.index, y=df['yhat_upper'],fill='tonexty',mode='lines',line_color='lightgrey', name='confidence_lev_up'))
        fig.add_traces(go.Scatter(x=df.index, y=df['y'], name='{}'.format(y), mode='lines+markers', marker=dict(size=5)))
        fig.add_traces(go.Scatter(x=df.loc[start_fit:end_fit].index, y=df['yhat'].loc[start_fit:end_fit], line_color='goldenrod',
                                  mode='lines',name='model fit'))
        fig.add_traces(go.Scatter(x=df.loc[end_fit:].index, y=df['yhat'].loc[end_fit:], line_color='darkblue', mode='markers',
                                  marker=dict(size=2), name='forecast'))
        fig.update_layout(showlegend=True,title={'text':label.replace('_', ' ') + ' for ' + region, 'xanchor': 'left'})
                         
        fig.update_xaxes(showgrid=False, gridwidth=1, gridcolor='gray')
        fig.update_yaxes(showgrid=False, gridwidth=1, gridcolor='gray')
        display(fig)
        fig.show()
        


forecast_widget=interactive(get_forecast, {'manual': True},
                              region=widgets.Dropdown(options=df_reg.keys(), value='Italy'), 
                              start_fit=widgets.DatePicker(value=pd.to_datetime(df_naz.index[0])), 
                              end_fit=widgets.DatePicker(value=pd.to_datetime(df_naz.index[-10])), 
                              label = widgets.Dropdown(options=trend_labels, value='nuovi_positivi'),
                              forecast_periods=50, smoothing=widgets.IntText(1),
                              continuous_update=False)
forecast_widget.children[-2].description = 'Launch Forecast'
forecast_box = widgets.HBox(forecast_widget.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))

forecast_tab.children = [forecast_box, forecast_out]

In [8]:
evo_tab = widgets.VBox()

evo_out = widgets.Output(layout = {
            'width': '100%',
            'height': '600px',
            'border': '1px solid black'
        })

def plt_region(regions,labels,log,relative_dates,cases_per_mln_people,plot_bars,show_grid,aggregate,apply_ma):    
    with evo_out:
        clear_output()
        if len(labels) == 0:
            labels = ['Rth'] 
        labels = list(labels)
        if len(regions) == 0:
            regions = ['Italy']
        regions = list(regions)  
        fig = go.Figure()
        mult = 1.
        for item in labels:
            if aggregate:
                if cases_per_mln_people: 
                    mult = 1e06/ita_populations.loc[regions, 'Popolazione'].sum()                
                temp = df_reg[regions[0]][item].copy() 
                for region in regions[1:]:
                    temp = temp.add(df_reg[region][item])
                if item == 'Rth':
                    if (len(regions[1:])) > 0:                        
                        temp = temp/len(regions[1:])
                temp = pd.DataFrame(temp)
                if relative_dates: temp = temp.loc[~(temp[item]==0)].reset_index(drop=True).iloc[:-1] 
                temp = temp.rolling(apply_ma).mean().shift(-0)
                if plot_bars:
                    fig.add_traces(go.Bar(x=temp.index, y=temp[item]*mult, name=item+'_'+'-'.join(regions)))
                else:
                    fig.add_traces(go.Scatter(x=temp.index, y=temp[item]*mult, name=item+'_'+'-'.join(regions)))

            else:
                for region in regions:
                    if cases_per_mln_people: 
                        mult = 1e06/ita_populations.loc[region, 'Popolazione']
                    df_reg[region].index = pd.to_datetime(df_reg[region].index)
                    temp = df_reg[region].copy() 
                    temp[item] = temp[item].rolling(apply_ma).mean().shift(-0)
                    if relative_dates: temp = temp.loc[~(temp[item]==0)].reset_index(drop=True).iloc[:-1] 
                        
                    if plot_bars:
                        fig.add_traces(go.Bar(x=temp.index, y=temp[item]*mult, name=item+'_'+region))
                    else:
                        fig.add_traces(go.Scatter(x=temp.index, y=temp[item]*mult, name=item+'_'+region))
        fig.update_layout(showlegend=True,
                          legend_orientation="h",
                          title='Regional Evolution')
        if log: fig.update_layout(yaxis_type="log")
        fig.show()
        
evo_widget = interactive(plt_region,
                         regions = widgets.SelectMultiple(description="regions",
                                                          options=list(df_reg.keys()), value=['Italy']), 
                         labels = widgets.SelectMultiple(description="fields",
                                                         options=orig_data_columns+extra_data_columns, value=['nuovi_positivi']),
                         log=False, 
                         relative_dates=False, 
                         cases_per_mln_people=False, 
                         plot_bars=True, 
                         show_grid=False,
                         aggregate=False, 
                         apply_ma=widgets.IntText(value=1, min=1, max=150))
evo_box = widgets.HBox(evo_widget.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))

evo_tab.children = [evo_box, evo_out]

In [ ]:
daily_tab = widgets.VBox()

daily_out = widgets.Output(layout = {
            'width': '100%',
            'height': '600px',
            'border': '1px solid black'
        })

def get_values_for_day(regions,labels,date,cases_per_mln_people,show_grid):
    with daily_out:
        clear_output()
        data_columns = orig_data_columns+extra_data_columns
        if len(regions) == 0:
            regions = ['Italy']
        regions = list(regions)    
        if len(labels) == 0:
            labels = [item for item in data_columns if ('daily' in item) & ('%' not in item)]
        labels = list(labels)
        mult = 1.
        fig = go.Figure()
        for region in regions:    
            if cases_per_mln_people: 
                mult = 1e06/ita_populations.loc[region, 'Popolazione']
            for item in labels: 
                df_reg[region].index = pd.to_datetime(df_reg[region].index)
            fig.add_traces(go.Bar(y=labels, x=df_reg[region][labels].loc[date]*mult, name=region, orientation='h'))
            fig.update_layout(showlegend=True,title='day ' + str(date.strftime("%m/%d/%Y")))
        fig.show()

default_regions = pd.DataFrame({reg: df_reg[reg]['Rth'].iloc[-1] for reg in df_reg.keys()}, index=[0]).T.sort_values(by=0, ascending=False).iloc[:10].index
    
daily_widget = interactive(get_values_for_day,
                           regions = widgets.SelectMultiple(description="regions",
                                                            options=list(df_reg.keys()), value=list(default_regions)),
                           labels = widgets.SelectMultiple(description="data",
                                                           options=orig_data_columns+extra_data_columns, 
                                                           value=['Rth']),
                           date=widgets.DatePicker(description='Pick a Date',
                                                   value=pd.to_datetime(df_prov.index.max())),
                           cases_per_mln_people=False, 
                           show_grid=False)
daily_box = widgets.HBox(daily_widget.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))

daily_tab.children = [daily_box, daily_out]

In [ ]:
italy_tab.children = [prov_tab, daily_tab, evo_tab, forecast_tab]
italy_tab.set_title(0, 'provinces')
italy_tab.set_title(1, 'daily cases')
italy_tab.set_title(2, 'evolution')
italy_tab.set_title(3, 'forecast')

In [ ]:
# add all components to main_tab page
main_tab.children=[italy_tab]#, download_tab]

In [ ]:
main_tab

In [ ]:
import requests
import io
url = "https://github.com/pcm-dpc/COVID-19/blob/master/dati-regioni/dpc-covid19-ita-regioni.csv"

In [ ]:
s=requests.get(url).text
c = pd.read_csv(io.StringIO(s))

In [ ]:
data = pd.read_csv("https://github.com/pcm-dpc/COVID-19/blob/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv") 

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv')

In [ ]:
df_reg